In [ ]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/cleaned_data.csv"

Mounted at /content/drive


In [ ]:
data= pd.read_csv(path)

In [ ]:
def create_df(data,Location):
    #Selecting the 7 key columns for country in dataset
    data=data[data['Location']==Location].copy()
    data=data[['Date','Total Cases','New Cases',
         'Total deaths','New deaths',
         "people vaccinated per hundred", 'people fully vaccinated per hundred',
          'New cases smoothed']].copy()
    #Convert to datetime
    #data2.date = pd.to_datetime(data2['Date'])
    data['Date'] = pd.DatetimeIndex(data['Date'])
    return data

In [ ]:
df=create_df(data,'India')

In [ ]:
# Rename columns as required by Prophet
#df = asia_train1.groupby('Date').sum("New cases smoothed").reset_index() \ .rename(columns={'Date': 'ds', "New cases smoothed": 'y'})
df = df.rename(columns={'Date': 'ds', 'New cases smoothed': 'y'})


In [ ]:
df['ds'] = pd.to_datetime(df['ds'])

In [ ]:
split_date='31-dec-2021'

asia_train1 = df.loc[df['ds'] <=split_date].copy()
asia_test1 = df.loc[df['ds'] >split_date].copy()


In [ ]:
asia_train1

,ds,y
69678,2020-01-30,0.000
69679,2020-01-31,0.000
69680,2020-02-01,0.000
69681,2020-02-02,0.000
69682,2020-02-03,0.000
...,...,...
70439,2022-03-01,10224.000
70440,2022-03-02,9140.143
70441,2022-03-03,8173.000
70442,2022-03-04,7376.143


In [ ]:
asia_train1 = df[['ds', 'y']]
asia_train1

,ds,y
69678,2020-01-30,0.000
69679,2020-01-31,0.000
69680,2020-02-01,0.000
69681,2020-02-02,0.000
69682,2020-02-03,0.000
...,...,...
70439,2022-03-01,10224.000
70440,2022-03-02,9140.143
70441,2022-03-03,8173.000
70442,2022-03-04,7376.143


In [ ]:
print(df.columns)

In [ ]:
#df = df.rename(columns={'Date': 'ds', 'New cases smoothed': 'y'})
#df['ds'] = pd.to_datetime(df['ds'])

In [ ]:
model = Prophet()
model.fit(asia_train1)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp56vw1ql8/h2dg4okp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp56vw1ql8/_9pzvmua.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=43387', 'data', 'file=/tmp/tmp56vw1ql8/h2dg4okp.json', 'init=/tmp/tmp56vw1ql8/_9pzvmua.json', 'output', 'file=/tmp/tmp56vw1ql8/prophet_model33c3vxz1/prophet_model-20230420041928.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:19:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:19:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)
test_predictions = forecast[-365:]['yhat']

In [ ]:
forecast[['ds', 'yhat']]


,ds,yhat
0,2020-01-30,20879.404710
1,2020-01-31,16642.915599
2,2020-02-01,11889.798208
3,2020-02-02,7314.937942
4,2020-02-03,1792.324556
...,...,...
1361,2023-10-22,163572.921379
1362,2023-10-23,163254.718131
1363,2023-10-24,162998.735566
1364,2023-10-25,162789.231602


In [ ]:
fig = plot_plotly(model, forecast)
fig.show()
